# Electric Motor Temperature Prediction
### Using Machine Learning - Rotor Temperature Detection

## Step 1: Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pickle
import warnings
warnings.filterwarnings('ignore')

print('Libraries imported successfully!')

## Step 2: Load Dataset

In [ ]:
df = pd.read_csv('pmsm_temperature_data.csv')
print('Shape:', df.shape)
df.head()

## Step 3: Exploratory Data Analysis

In [ ]:
print('Dataset Info:')
df.info()
print('\nStatistical Summary:')
df.describe()

In [ ]:
# Check missing values
print('Missing Values:')
print(df.isnull().sum())

In [ ]:
# Correlation heatmap
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Feature Correlation Heatmap')
plt.tight_layout()
plt.show()

In [ ]:
# Distribution of target variable (pm = permanent magnet / rotor temperature)
plt.figure(figsize=(8,5))
sns.histplot(df['pm'], kde=True, color='steelblue')
plt.title('Distribution of Rotor Temperature (pm)')
plt.xlabel('Temperature')
plt.show()

## Step 4: Data Preprocessing

In [ ]:
# Drop profile_id (session identifier, not a feature)
if 'profile_id' in df.columns:
    df = df.drop('profile_id', axis=1)

# Define features and target
target = 'pm'  # rotor/permanent magnet temperature
features = [col for col in df.columns if col != target]

X = df[features]
y = df[target]

print('Features:', features)
print('Target:', target)
print('X shape:', X.shape)
print('y shape:', y.shape)

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('Train size:', X_train.shape)
print('Test size:', X_test.shape)

## Step 5: Model Training

In [ ]:
# Train Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train_scaled, y_train)
print('Model training complete!')

## Step 6: Model Evaluation

In [ ]:
y_pred = model.predict(X_test_scaled)

mse  = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f'MSE  : {mse:.4f}')
print(f'RMSE : {rmse:.4f}')
print(f'MAE  : {mae:.4f}')
print(f'R²   : {r2:.4f}')

In [ ]:
# Actual vs Predicted
plt.figure(figsize=(10,5))
plt.plot(y_test.values[:200], label='Actual', color='blue')
plt.plot(y_pred[:200], label='Predicted', color='orange', linestyle='--')
plt.title('Actual vs Predicted Rotor Temperature')
plt.xlabel('Samples')
plt.ylabel('Temperature')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Feature Importance
importances = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
plt.figure(figsize=(8,5))
importances.plot(kind='bar', color='steelblue')
plt.title('Feature Importance')
plt.tight_layout()
plt.show()

## Step 7: Save Model and Scaler

In [ ]:
# Save model
with open('model.save', 'wb') as f:
    pickle.dump(model, f)

# Save scaler/transformer
with open('transform.save', 'wb') as f:
    pickle.dump(scaler, f)

print('Model and transformer saved successfully!')